# Manipulating Data With NDArray

NDArray用于完成对数组元素的操作，它是MXNet中存储和变换数据的主要工具，与Numpy相比的主要优势有：

1.支持CPU/GPU/分布式下的异步计算

2.支持自动微分，因此它很适合用来开发机器学习深度学习应用

## NDArray的基本用法

In [39]:
import mxnet as mx
from mxnet import nd
# 设置随机种子 使得每次随机生成的数都是一样的
mx.random.seed(1)

In [40]:
a = nd.random_normal(shape=(4,4))
a


[[  1.12877361e-01  -1.30644417e+00  -1.07135750e-01  -2.63099265e+00]
 [ -5.73584773e-02   3.13484162e-01  -5.76510906e-01  -1.11059952e+00]
 [  5.79607189e-01  -2.28995964e-01   1.04484284e+00   8.12436819e-01]
 [  6.01003528e-01   2.03541629e-02  -8.78813088e-01   8.88380397e-04]]
<NDArray 4x4 @cpu(0)>

In [41]:
b = nd.empty((3,4))
b


[[  0.00000000e+00   0.00000000e+00  -1.93917308e-22   4.57229676e-41]
 [  1.50539812e-38   0.00000000e+00   8.60128526e+27   4.57229676e-41]
 [  9.71215350e+27   4.57229676e-41   2.19902965e-40  -3.76175546e-37]]
<NDArray 3x4 @cpu(0)>

In [42]:
c = nd.zeros((2,2))
c


[[ 0.  0.]
 [ 0.  0.]]
<NDArray 2x2 @cpu(0)>

In [43]:
d = nd.random_normal(0,1,shape=(4,3))
d


[[-0.73376489 -0.76703477 -1.12628222]
 [ 1.38539445  1.92117143  1.50441384]
 [-0.85433006 -2.54523706 -0.51121545]
 [ 0.15673178  0.03484428  0.77970827]]
<NDArray 4x3 @cpu(0)>

## In-Place 操作(就地操作)

有时候我们不想对每个计算后的结果都分配一个全新的内存，比如当我们计算y=x+y的时候，程序内部会首先去除对y原先地址的引用(我们称之为去引用),之后会对y计算后的新的结果分配一块新的内存，可以通过python的id()函数拿到元素值的地址来证明这一点。

In [45]:
x = nd.ones((3,4))
y = nd.random_normal(0,1,shape=(3,4))
print("id(y)", id(y))
y = x + y
print("id(y)", id(y))

id(y) 140142327586488
id(y) 140142327586432


这种结果其实我们是不愿意看到的，因为我们不原意系统一直在分配一些不必要的内存，这会产生极大地浪费。在ML中，我们有数百兆的参数，并且要在一秒内对其更新数次，通常我们希望的是能对这些参数都进行就地更新。第二点是说，通常我们可以通过定义多个变量来指向相同的参数，如果我们不进行就地更新，这可能会造成内存泄漏，结果就是我们可能会引用旧的参数。

我们可以通过切片操作来完成就地操作：

In [46]:
print("id(y)", id(y))
y[:] = x + y
print("id(y)", id(y))

id(y) 140142327586432
id(y) 140142327586432


这种方式虽然最终完成了就地操作，但在将结果分配给y[:]之前仍然会开辟一个小的缓冲区来存储x+y的结果，为了避免这个缓冲区的产生，我们可以直接调用底层的ndarray操作，即elemwise_add来避免缓冲区的产生，另外，每一个操作符都支持这种操作

In [49]:
nd.elemwise_add(x, y, out=y)


[[ 12.69358921   2.09899282   8.41394424   4.00772476]
 [  6.38052511   5.60571289   4.5248251    5.6987009 ]
 [  9.67530155   5.95555782  11.21280861   5.22764254]]
<NDArray 3x4 @cpu(0)>

如果我们不需要再重复使用x，那么我们可以将计算后的结果分配给x自己，这在MXNet中通常有两种实现方式

1.通过切片操作符x[:]

2.通过使用+=操作符

In [50]:
print('id(x):', id(x))
x += y
x
print('id(x):', id(x))

id(x): 140142328613072
id(x): 140142328613072


## Slicing

In [52]:
x


[[ 19.54038429   3.64848924  13.12091637   6.51158714]
 [ 10.07078743   8.90856934   7.28723764   9.04805183]
 [ 15.0129528    9.43333626  17.31921387   8.34146404]]
<NDArray 3x4 @cpu(0)>

In [60]:
x[2:3]


[[ 15.0129528    9.43333626  17.31921387   8.34146404]]
<NDArray 1x4 @cpu(0)>

In [61]:
x[1,2] = 666
x


[[  19.54038429    3.64848924   13.12091637    6.51158714]
 [  10.07078743    8.90856934  666.            9.04805183]
 [  15.0129528     9.43333626   17.31921387    8.34146404]]
<NDArray 3x4 @cpu(0)>

In [62]:
x[1:2,1:3]


[[   8.90856934  666.        ]]
<NDArray 1x2 @cpu(0)>

In [64]:
x[1:2,1:3] = 555
x


[[  19.54038429    3.64848924   13.12091637    6.51158714]
 [  10.07078743  555.          555.            9.04805183]
 [  15.0129528     9.43333626   17.31921387    8.34146404]]
<NDArray 3x4 @cpu(0)>

## Broadcasting

In [67]:
x = nd.ones((3,3))
print("x:", x)
y = nd.arange(3)
print("y:", y)
print("x+y:", x+y)

x: 
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @cpu(0)>
y: 
[ 0.  1.  2.]
<NDArray 3 @cpu(0)>
x+y: 
[[ 1.  2.  3.]
 [ 1.  2.  3.]
 [ 1.  2.  3.]]
<NDArray 3x3 @cpu(0)>


In [69]:
y = y.reshape((3,1))
print("y:", y)
print("x+y:", x+y)

y: 
[[ 0.]
 [ 1.]
 [ 2.]]
<NDArray 3x1 @cpu(0)>
x+y: 
[[ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]]
<NDArray 3x3 @cpu(0)>


## Convert from MXNet NDArray to Numpy 

In [70]:
type(x)

mxnet.ndarray.ndarray.NDArray

In [74]:
a = x.asnumpy()
type(a)

numpy.ndarray

## Managing context

MXNet支持在NDArray中设置不同的context从而在不同的硬件设备环境上运行不同的NDArray，我们首先在一个CPU上创建一个NDArray：

In [88]:
z = nd.ones(shape=(3,3), ctx=mx.cpu(0))
z


[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @cpu(0)>

In [90]:
x_cpu = z.copyto(mx.cpu(1))
x_cpu


[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @cpu(1)>

In [83]:
z + x_cpu 


[[ 2.  2.  2.]
 [ 2.  2.  2.]
 [ 2.  2.  2.]]
<NDArray 3x3 @cpu(0)>

In [91]:
print(x_cpu.context)
print(z.context)

cpu(1)
cpu(0)


## 使用as_in_context而不是使用copyto

当我们使用copyto拷贝一个ctx下的NDArray到另一个ctx下时，不管目标ctx是否与当前ctx相同，都会产生一份新的拷贝，即分配一块新的内存，而使用as_in_context的好处是说当target ctx与source ctx相同时，我们不产生一份新的拷贝，而只返回当前的NDArray。可以通过下面的例子来论述：

如果cpu(0)上已经有e，仍然要在通过e在cpu(0)上创建f时，会分配一块新的内存：

In [98]:
e = nd.ones(shape=(1,1), ctx=mx.cpu(0))
f = e.copyto(mx.cpu(0))
print(f is e)
print(id(e))
print(id(f))

False
140142331324792
140142328626816


而使用as_in_context时，会直接进行就地操作

In [99]:
e = nd.ones(shape=(1,1), ctx=mx.cpu(0))
f = e.as_in_context(mx.cpu(0))
print(f is e)
print(id(e))
print(id(f))

True
140142979635912
140142979635912


所以在通常情况下，我们都使用as_in_context而不使用copyto

## 参考文献

1.[Manipulate data the MXNet way with ndarray](https://github.com/zackchase/mxnet-the-straight-dope/blob/master/chapter01_crashcourse/ndarray.ipynb)